In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import re
from tqdm.notebook import trange, tqdm
import classifier_utils as cu
import utils
import plotly.figure_factory as ff


from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [15]:
def predict_list(user_tweets, model, limit=100):
    user_scores = {}
    user_preds = {}
    i = 0
    for uid, tweets in tqdm(user_tweets.items()):
        if len(tweets) < 500:
            continue
        texts = tweets #[t.text for t in tweets]
        probs = model.predict_proba(texts)[:, 1]
        user_scores[uid] = probs.mean(axis=0)
        user_preds[uid] = probs
        i += 1
        if i > limit:
            break
    return user_scores, user_preds

def compute_harassment_frac(user_tweets, threshold=0.5):
    user_scores = {}
    for uid, tweet_probs in user_tweets.items():
        user_scores[uid] = (tweet_probs > threshold).sum() / len(tweet_probs)
    return user_scores

In [94]:
target_users_df = pd.read_csv('../data/target_users.csv', index_col=0)

In [98]:
len(target_users_df)

270

In [58]:
users = target_users_df[[
#     'harassing_users_final_model_gboost_toxic_comments_identity_hate',
#     'harassing_users_final_model_gboost_toxic_comments_insult',
#     'harassing_users_final_model_gboost_toxic_comments_obscene',
#     'harassing_users_final_model_gboost_toxic_comments_severe_toxic',
#     'harassing_users_final_model_gboost_toxic_comments_threat',
    'harassing_users_final_model_gboost_toxic_comments_toxic',
#     'harassing_users_final_model_gboost_hate_offensive_Hate',
#     'harassing_users_final_model_gboost_hate_offensive_Offensive',
#     'harassing_users_final_model_gboost_hate_abuse_abusive',
#     'harassing_users_final_model_gboost_hate_abuse_hateful'
]].values.flatten()

all_users = []
for s in users:
    if s is not '[]':
        all_users.extend([u for u in re.sub("\[|\]|\'|\,", '', s).split(' ') if len(u) > 0])
all_users = np.unique(all_users)

In [21]:

model_files = [
    'final_model_toxic_comments_identity_hate.p',
    'final_model_toxic_comments_insult.p',
    'final_model_toxic_comments_obscene.p',
    'final_model_toxic_comments_severe_toxic.p',
    'final_model_toxic_comments_threat.p',
    'final_model_toxic_comments_toxic.p',
    
    'final_model_gboost_toxic_comments_identity_hate.p',
    'final_model_gboost_toxic_comments_insult.p',
    'final_model_gboost_toxic_comments_obscene.p',
    'final_model_gboost_toxic_comments_severe_toxic.p',
    'final_model_gboost_toxic_comments_threat.p',
    'final_model_gboost_toxic_comments_toxic.p',
    
    'final_model_gboost_hate_offensive_Hate.p',
    'final_model_gboost_hate_offensive_Offensive.p',
    'final_model_gboost_hate_abuse_abusive.p',
    'final_model_gboost_hate_abuse_hateful.p',
]
models = []
for file in model_files:
    with open(file, 'rb') as f:
        models.append((file, pickle.load(f)))

In [27]:
harasser_tweets = utils.load_user_tweets_by_name('../data/harassers_from_targetted/', all_users)

KeyboardInterrupt: 

In [24]:
results = {}
for name, m in models:
    results[name] = predict_list(harasser_tweets, m, limit=1000)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'becau'] not in stop_words.



In [25]:
with open('predict_harasser_scores.p', 'wb') as f:
    pickle.dump(results, f)

In [9]:
with open('final_model_gboost_toxic_comments_toxic.p', 'rb') as f:
    model = pickle.load(f)

In [16]:
user_scores, user_preds = predict_list(harasser_tweets, model, limit=500)

In [28]:
len(user_scores), len(results)

(501, 16)

In [32]:
with open('predict_blocklists_res2.p', 'rb') as f:
    results_control = pickle.load(f)

In [34]:
results_control.keys()

dict_keys(['final_model_toxic_comments_identity_hate.p', 'final_model_toxic_comments_insult.p', 'final_model_toxic_comments_obscene.p', 'final_model_toxic_comments_severe_toxic.p', 'final_model_toxic_comments_threat.p', 'final_model_toxic_comments_toxic.p', 'final_model_gboost_toxic_comments_identity_hate.p', 'final_model_gboost_toxic_comments_insult.p', 'final_model_gboost_toxic_comments_obscene.p', 'final_model_gboost_toxic_comments_severe_toxic.p', 'final_model_gboost_toxic_comments_threat.p', 'final_model_gboost_toxic_comments_toxic.p', 'final_model_gboost_hate_offensive_Hate.p', 'final_model_gboost_hate_offensive_Offensive.p', 'final_model_gboost_hate_abuse_abusive.p', 'final_model_gboost_hate_abuse_hateful.p'])

In [39]:
control_scores, control_preds = results_control['final_model_gboost_toxic_comments_toxic.p']['control2']

In [33]:
harasser_scores, harasser_preds = results['final_model_gboost_toxic_comments_toxic.p']

In [59]:
len(all_users), len(harasser_scores)

(1447, 1001)

In [44]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=list(compute_harassment_frac(control_preds).values()))
)
fig.add_trace(
    go.Histogram(x=list(compute_harassment_frac(harasser_preds).values()))
)
fig.show()

In [93]:
fig = go.Figure()


user_scores = list(compute_harassment_frac(control_preds, threshold=0.7).values())
# user_scores = list(control_scores.values())
# bin_edges = np.histogram_bin_edges(user_scores, bins='fd', range=(0.0, 0.5))

# weights = np.ones_like(user_scores)/(len(user_scores))
# hist, _= np.histogram(user_scores, bins=bin_edges, weights=weights)
# fig.add_trace(
#     go.Scatter(
#         x=bin_edges, y=hist, 
#         name='Random',
#         mode='lines', 
#     )
# )

fig.add_trace(
    go.Histogram(x=user_scores, name='Random',histnorm='probability')
)

user_scores = list(compute_harassment_frac(harasser_preds, threshold=0.5).values())
# user_scores = list(harasser_scores.values())
# bin_edges = np.histogram_bin_edges(user_scores, bins='fd', range=(0.0, 0.5))

# weights = np.ones_like(user_scores)/(len(user_scores))
# hist, _= np.histogram(user_scores, bins=bin_edges, weights=weights)
# fig.add_trace(
#     go.Scatter(
#         x=bin_edges, y=hist, 
#         name='Harasser',
#         mode='lines', 
#     )
# )

fig.add_trace(
    go.Histogram(x=user_scores, name='Harasser', histnorm='probability')
)


fig.update_layout(barmode='overlay')
fig.update_layout(
    autosize=False,
    width=350,
    height=200,
    legend=dict(x=0.65, y=0.95),
    margin=dict(l=20, r=20, t=20, b=20)
)

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.65)



In [ ]:

fig = make_subplots(rows=4, cols=2, subplot_titles=display_names)
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    legend=dict(x=0.6, y=0),
    margin=dict(l=20, r=20, t=20, b=20)
)
showlegend=True
c = 0
for model_name in model_names:
    res = results[model_name]
    for i in range(len(user_list_names)):
        list_name = user_list_names[i]
        user_scores = list(res[list_name][0].values())
        bin_edges = np.histogram_bin_edges(user_scores, bins='auto', range=(0.0, 0.5))
        hist, _= np.histogram(user_scores, bins=bin_edges, density=True)
        fig.add_trace(
            go.Scatter(
                x=bin_edges, y=hist, 
                name=list_display_names[i],
                mode='lines', 
                legendgroup="group", 
                showlegend=showlegend,
                line=dict(color=colors[i]),
            ),
            col= c % 2 + 1, row= c // 2 + 1
        )
    c += 1
    showlegend=False
fig.show()